# Modelagem do problema do Titanic

 - Realização do tratamento de outliers (se necessário); 
 - Criação prática das features (se necessário);
 - Normalização; (perguntar)
 - Execução do Encoder;
 - Treinamento, Validação e Teste do modelo;

##### Importando as bibliotecas necessárias e dataset processado

In [60]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Carregando o modelo processado na etapa anterior (pré processamento)
model_path = r'C:\Users\Fernando Costa\Desktop\Arquivos Lucas\DS\Projetos GITHUB\Titanic\df_model.parquet'
df_model = pd.read_parquet(model_path)

# Definindo as colunas categóricas do dataset
col_categoricas = ['Pclass', 'Sex', 'Cabin', 'Embarked']

## Outliers e Novas Features

Por conta da limitação desta base de dados, apenas será utilizada a nova feature implementada na etapa de Pre Processamento e os outliers ignorados.

## Segregando Dados

Implementando a segregação dos dados de treino e dados de teste

In [61]:
X = df_model.drop(columns='Survived')  
y = df_model['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Encoding

Importando e segregando dados para utilização do encoder

In [62]:
# Inicializar o encoder
catboost_encoder = CatBoostEncoder()

# Codificando apenas as colunas categóricas 'Pclass', 'Sex', 'Cabin', 'Embarked'
X_train2encode = X_train[col_categoricas]
y_train2encode = y_train

# Ajustar o encoder nos dados de treino
catboost_encoder.fit(X_train2encode, y_train2encode)

CatBoostEncoder(cols=['Pclass', 'Sex', 'Cabin', 'Embarked'])

Transformando os dados de treino e teste

In [63]:
X_train_encoded = catboost_encoder.transform(X_train[col_categoricas])
X_test_encoded = catboost_encoder.transform(X_test[col_categoricas])

Renomeando as colunas codificadas

In [64]:
rename_dict = {}
for original_col, encoded_col in zip(col_categoricas, col_categoricas):
    rename_dict[encoded_col] = f"{original_col}_encoded"

X_train_encoded.rename(columns=rename_dict, inplace=True)
X_test_encoded.rename(columns=rename_dict, inplace=True)

Obtendo a lista das colunas categóricas codificadas:

In [65]:
coded_col_categoricas = X_train_encoded.columns

Adicionando as colunas categóricas ao X_treino e X_teste original

In [66]:
X_train_final = pd.concat([X_train, X_train_encoded], axis=1)
X_test_final = pd.concat([X_test, X_test_encoded], axis=1)

X_train_final = X_train_final.drop(columns=col_categoricas)
X_test_final = X_test_final.drop(columns=col_categoricas)

## Treinamento, Validação e Teste do Modelo

Para fins de treinamento, utilizaremos dois algoritmos: RandomForest e Regressão Logística.

In [67]:
# Inicializando e treinando o modelo RandomForest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_final, y_train)

# Inicializando e treinando o modelo de Regressão Logística
lr_model = LogisticRegression()
lr_model.fit(X_train_final, y_train)

c:\Users\Fernando Costa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Para Validar os modelos treinados, utilizaremos a métrica roc-auc

In [68]:
# Fazendo as previsões
rf_predictions = rf_model.predict_proba(X_test_final)[:, 1]
lr_predictions = lr_model.predict_proba(X_test_final)[:, 1]

# Avaliando o desempenho com ROC-AUC
rf_auc = roc_auc_score(y_test, rf_predictions)
lr_auc = roc_auc_score(y_test, lr_predictions)

print(f'Random Forest ROC-AUC: {rf_auc:.4f}')
print(f'Regressão Logística ROC-AUC: {lr_auc:.4f}')

Random Forest ROC-AUC: 0.8854
Regressão Logística ROC-AUC: 0.8699


## Testando Modelos na "Prática"

Importando a base de dados de "Teste real", aplicando o processamento realizado anteriormente ao df_model

In [69]:
# Importando a função criada a partir do pré processamento:
from PP_titanic import PP_titanic_function

aval_path = r'C:\Users\Fernando Costa\Desktop\Arquivos Lucas\DS\Projetos GITHUB\Titanic\Base_dados\test.csv'
X_aval = pd.read_csv(aval_path)

In [70]:
# Breve teste do df de avaliação. Checando valores nulos
display(X_aval.info())
display(X_aval.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


None

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Foi encontrado um valor nulo, como a coluna Fare é do tipo float64, e não houve esse problema com o df original, preencheremos com -1

In [71]:
X_aval = X_aval.fillna(-1)

In [72]:
# Aplicando o processamento ao novo df
X_aval = PP_titanic_function(X_aval)

Utilizando o Encoder ao df de avaliação:

In [73]:
# Aplicando o catboost_encoder
X_aval_encoded = catboost_encoder.transform(X_aval[col_categoricas])

# Renomeando as colunas
X_aval_encoded.rename(columns=rename_dict, inplace=True)

# Unindo o df original ao codificado
X_aval = pd.concat([X_aval, X_aval_encoded], axis=1)

# Excluindo as colunas categoricas originais
X_aval = X_aval.drop(columns=col_categoricas)

Realizando novas predições, mas agora em relação ao novo df de avaliação:

In [74]:
# Fazendo as previsões
rf_predictions = rf_model.predict_proba(X_aval)[:, 1]
lr_predictions = lr_model.predict_proba(X_aval)[:, 1]

# Como não há forma de testar esses dados diretamente, pois não existe y_aval, finalizamos por aqui.

## Cross Validation e Pipeline (Não implementado)

Nesta etapa, será definida as diferentes divisões da base de dados, além da implementação do encoding (evitando data leakage) para cada um dos dois algoritmos a serem implementados: RandomForest e LogisticRegression

In [75]:
# Separando inicialmente as variáveis preditoras e a target
X = df_model.drop(columns='Survived')
y = df_model['Survived']

# Configurando o Cross-Validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Os pipelines serão criados com o CatBoost Encoder já embutido
# Criando o pipeline com Logistic Regression
pipeline_logreg = Pipeline([
    ('catboost_encoder', CatBoostEncoder(cols=col_categoricas, random_state=42)),  
    ('classifier', LogisticRegression(solver='liblinear'))                           
])

# Criando o pipeline com RandomForestClassifier
pipeline_rf = Pipeline([
    ('catboost_encoder', CatBoostEncoder(cols=col_categoricas, random_state=42)), 
    ('classifier', RandomForestClassifier(random_state=42))                         
])

# Passo 6: Realizar o Cross-Validation e calcular as métricas (ROC-AUC)
scores_logreg = cross_val_score(pipeline_logreg, X, y, cv=cv, scoring='roc_auc')
scores_rf = cross_val_score(pipeline_rf, X, y, cv=cv, scoring='roc_auc')

# Passo 7: Mostrar os resultados
print(f'ROC-AUC para Regressão Logística em cada fold: {scores_logreg}')
print(f'ROC-AUC médio para Regressão Logística: {scores_logreg.mean()}')

print(f'\nROC-AUC para Random Forest em cada fold: {scores_rf}')
print(f'ROC-AUC médio para Random Forest: {scores_rf.mean()}')

ROC-AUC para Regressão Logística em cada fold: [0.85250329 0.82927807 0.82199198 0.82486631 0.87760936]
ROC-AUC médio para Regressão Logística: 0.8412498035802782

ROC-AUC para Random Forest em cada fold: [0.88412385 0.87794118 0.82800802 0.85147059 0.86531046]
ROC-AUC médio para Random Forest: 0.861370819459524
